# Tutoriel MOPITT - Extraction de données et visualisation
***

**Tutoriel :** Ce tutoriel fournit un guide étape par étape pour extraire des données du Portail de Données Ouvertes de l'Agence spatiale canadienne. </br>
**Mission et Instrument :** MOPITT  
**Cible Astronomique :** Mesure du monoxyde de carbone dans la troposphère  
**Exigences du système :** Python 3.8 ou ultérieur  
**Niveau du tutoriel :** Basique  

Le but de ce tutoriel est d'aider à démontrer le processus d'accès aux données MOPITT, de les préparer pour utilisation, et de montrer quelques analyses de données de base et visualisation en utilisant les données MOPITT à bord de Terra. Ce tutoriel utilise des mesures de monoxyde de carbone dans la troposphère comme données d'analyse.

### Importation des bibliothèques requises et chargement des données

In [ ]:
import pandas as pd
import numpy as np
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

In [ ]:
# Load data from the Open Data Portal
df = pd.DataFrame()
for day in range(1,4):
    try:
        df_tmp = pd.read_csv(f'https://data.asc-csa.gc.ca/users/OpenData_DonneesOuvertes/pub/MOPITT/2019/MOP02J-201901{str(day).zfill(2)}-L2V18.0.3.csv', engine='python')
    except Exception:
        continue
    df = pd.concat([df, df_tmp])

In [ ]:
# Display the first rows to make sure everything is good
df.head()

In [ ]:
# Get the geo data
geometry = [Point(xy) for xy in zip(df[' Longitude'], df['# Latitude'])]
gdf = GeoDataFrame(df[' COTotalColumn'], geometry=geometry)
gdf.head()

In [ ]:
# Filter the concentration of carbon monoxide
d = gdf[' COTotalColumn'].values
t = d.mean() + (5 * d.std())
gdf_2 = gdf[gdf[' COTotalColumn'] < t]

### Affichage de la concentration du monoxyde de carbone

In [ ]:
# Display the concentration of carbon monoxide
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdf_2.plot(ax=world.plot(figsize=(14, 9)), column=' COTotalColumn', cmap='Greys', alpha=0.5);

### Deuxième scénario - Chargement de données

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

#lecture des données / Read data
data = np.loadtxt('https://data.asc-csa.gc.ca/users/OpenData_DonneesOuvertes/pub/MOPITT/2018/MOP02J-20181206-L2V18.0.3.csv', delimiter=',', skiprows=0).T 

lat,long = data[0:2]    #Latitude + Longitude
COTotalColumn = data[2] #CO total
COMR_Surface = data[3]  #CO Mixing Ratio Surface
COMR_Profile = data[4:-1]   #CO Mixing Ratio profile (900hPa-100hPa)
RetrievedSurfaceTemperature = data[-1]  #Retrieved Surface Temperature

### Affichage de la concentration du monoxyde de carbone

In [ ]:
# VISUALISATION GRAPHIQUE DE LA COLONNE TOTALE DU CO SUR CARTE DU MONDE AVEC BASEMAP
# DATA VISUALIZATION OF CO TOTAL COLUMN ON WORLD MAP USING BASEMAP
m = Basemap(projection='cyl', resolution='l',
            llcrnrlat=-90, urcrnrlat=90,
            llcrnrlon=-180, urcrnrlon=180)
m.drawcoastlines(linewidth=0.5)
m.drawparallels(np.arange(-90, 91, 45))
m.drawmeridians(np.arange(-180, 180, 45), labels=[True,False,False,True])
sc = m.scatter(long, lat, c=COTotalColumn, s=1, cmap=plt.cm.jet, edgecolors=None, linewidth=0)
cb = m.colorbar()
cb.set_label('mol/cm^2')
                        
plt.title('CONCENTRATION OF CARBON MONOXIDE')
fig = plt.gcf()
plt.show()